# A/B test for ENIAC

## 1. Loading data

In [172]:
import pandas as pd
from scipy import stats
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [173]:
url = "https://drive.google.com/file/d/1Ah_4aXosUyFj3CF_3NgTVZchtJlKiXoQ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id=" + url.split("/")[-2]
eniac_a = pd.read_csv(path)

url = "https://drive.google.com/file/d/1ISG8oA1zilVIaFIEyzdJ2S3zsEjSMdfJ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id=" + url.split("/")[-2]
eniac_b = pd.read_csv(path)

url = "https://drive.google.com/file/d/1BPUvd1xXkRWRGfv6UvPm5OoFfoTz_w1_/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id=" + url.split("/")[-2]
eniac_c = pd.read_csv(path)

url = "https://drive.google.com/file/d/1LY3qHH7MBPPqlGh-zNIfrHBLw8kj1BFQ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id=" + url.split("/")[-2]
eniac_d = pd.read_csv(path)

## 2. Exploring data

In [174]:
eniac_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57 entries, 0 to 56
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Element ID            57 non-null     int64 
 1   Tag name              57 non-null     object
 2   Name                  57 non-null     object
 3   No. clicks            57 non-null     int64 
 4   Visible?              57 non-null     bool  
 5   Snapshot information  2 non-null      object
dtypes: bool(1), int64(2), object(3)
memory usage: 2.4+ KB


In [175]:
eniac_b.loc[eniac_b['Name'] == 'mySidebar', 'Snapshot information'].str.extract(r'\s(\d+)\svisits').values[0][0]

'24747'

In [176]:
eniac_b

,Element ID,Tag name,Name,No. clicks,Visible?,Snapshot information
0,48,h1,ENIAC,236,True,Homepage Version B - red SHOP NOW • https://eniac.com/index-b.php
1,25,div,mySidebar,304,True,"created 2021-10-27 • 14 days 0 hours 34 mins • 24747 visits, 22592 clicks"
2,4,a,Mac,268,True,NaN
3,69,a,iPhone,260,True,NaN
4,105,a,Accessories,1214,True,NaN
5,36,a,Chargers & Cables,1259,False,NaN
6,99,a,iPhone Accessories,1237,False,NaN
7,68,a,Watch Accessories,1221,False,NaN
8,13,a,Mac Accessories,1210,False,NaN
9,15,a,AirTag,195,False,NaN


The dataset has the number of clicks in the [No. clicks] column, with the [Name] column with value 'SHOP NOW'.

Total number of clicks in the sum of the [No. clicks] column.

## 3. Creating contingent table

In [207]:
click = [
          eniac_a.loc[eniac_a['Name'] == 'SHOP NOW', 'No. clicks'].values[0]
        , eniac_d.loc[eniac_d['Name'] == 'SEE DEALS', 'No. clicks'].values[0]]
no_click = [
      int(eniac_a.loc[eniac_a['Name'] == 'mySidebar', 'Snapshot information'].str.extract(r'\s(\d+)\svisits').values[0][0]) - click[0]
    , int(eniac_d.loc[eniac_d['Name'] == 'mySidebar', 'Snapshot information'].str.extract(r'\s(\d+)\svisits').values[0][0]) - click[1]]

clicks2 = pd.DataFrame([click, no_click],
                                  columns = ["a", "c"],
                                  index = ["click", "no-click"])

clicks2 = clicks2/10
statistic, pvalue, dof, freq = stats.chi2_contingency(clicks2)
freq = pd.DataFrame(freq, columns = ['a', 'd'], index = ['click', 'no-click'])
pvalue

0.00022379716518278195

In [177]:
click = [
          eniac_a.loc[eniac_a['Name'] == 'SHOP NOW', 'No. clicks'].values[0]
        , eniac_b.loc[eniac_b['Name'] == 'SHOP NOW', 'No. clicks'].values[0]
        , eniac_c.loc[eniac_c['Name'] == 'SEE DEALS', 'No. clicks'].values[0]
        , eniac_d.loc[eniac_d['Name'] == 'SEE DEALS', 'No. clicks'].values[0]]
no_click = [
      int(eniac_a.loc[eniac_a['Name'] == 'mySidebar', 'Snapshot information'].str.extract(r'\s(\d+)\svisits').values[0][0]) - click[0]
    , int(eniac_b.loc[eniac_b['Name'] == 'mySidebar', 'Snapshot information'].str.extract(r'\s(\d+)\svisits').values[0][0]) - click[1]
    , int(eniac_c.loc[eniac_c['Name'] == 'mySidebar', 'Snapshot information'].str.extract(r'\s(\d+)\svisits').values[0][0]) - click[2]
    , int(eniac_d.loc[eniac_d['Name'] == 'mySidebar', 'Snapshot information'].str.extract(r'\s(\d+)\svisits').values[0][0]) - click[3]]

clicks = pd.DataFrame([click, no_click],
                                  columns = ["a", "b", "c", 'd'],
                                  index = ["click", "no-click"])
clicks_ctr = clicks.copy()
clicks_ctr.loc['CTR/10000'] = clicks_ctr.iloc[0] / (clicks_ctr.iloc[1] + clicks_ctr.iloc[0]) * 10000
clicks_ctr

,a,b,c,d
click,512.000,281.000,527.000,193.000
no-click,24814.000,24466.000,24349.000,25040.000
CTR/10000,202.164,113.549,211.851,76.487


## 4. Calculating statistics

Chi-squared test:

In [178]:
statistic, pvalue, dof, freq = stats.chi2_contingency(clicks)
freq = pd.DataFrame(freq, columns = ['a', 'b', 'c', 'd'], index = ['click', 'no-click'])

In [179]:
print(f'statistic: {statistic}')
print(f'pvalue: {pvalue}')
print(f'dof: {dof}')
print(f'freq: ')
freq


statistic: 224.01877488058412
pvalue: 2.7161216607868712e-48
dof: 3
freq: 


,a,b,c,d
click,382.486,373.742,375.690,381.082
no-click,24943.514,24373.258,24500.310,24851.918


## 5. Conclusion 1

We reject the H<sub>0</sub> that the site versions have the same CTR.

We also need to figure out which site has the highest CTR and whether or not it is statistically significant.

In [180]:
clicks_ctr

,a,b,c,d
click,512.000,281.000,527.000,193.000
no-click,24814.000,24466.000,24349.000,25040.000
CTR/10000,202.164,113.549,211.851,76.487


## 6. Statistics for version pairs

We create tests for all pairs. But we can already see from the CTRs that we especially need to compare a and c versions.

In [181]:
clicks_ab = clicks.copy()[['a', 'b']]
chisq_ab, pvalue_ab, df_ab, expected_ab = stats.chi2_contingency(clicks_ab)
print(f'p-value for ab is :{pvalue_ab:.3f}')

p-value for ab is :0.000


In [182]:
clicks_ac = clicks.copy()[['a', 'c']]
chisq_ac, pvalue_ac, df_ac, freq_ac = stats.chi2_contingency(clicks_ac)
print(f'p-value for ac is :{pvalue_ac:.3f}')

p-value for ac is :0.465


In [183]:
clicks_ad = clicks.copy()[['a', 'd']]
chisq_ad, pvalue_ad, df_ad, expected_ad = stats.chi2_contingency(clicks_ad)
print(f'p-value for ad is :{pvalue_ad:.3f}')

p-value for ad is :0.000


In [184]:
clicks_bc = clicks.copy()[['b', 'c']]
chisq_bc, pvalue_bc, df_bc, expected_bc = stats.chi2_contingency(clicks_bc)
print(f'p-value for bc is :{pvalue_bc:.3f}')

p-value for bc is :0.000


In [185]:
clicks_bd = clicks.copy()[['b', 'd']]
chisq_bd, pvalue_bd, df_bd, expected_bd = stats.chi2_contingency(clicks_bd)
print(f'p-value for bd is :{pvalue_bd:.3f}')

p-value for bd is :0.000


In [186]:
clicks_cd = clicks.copy()[['c', 'd']]
chisq_cd, pvalue_cd, df_cd, expected_cd = stats.chi2_contingency(clicks_cd)
print(f'p-value for cd is :{pvalue_cd:.3f}')

p-value for cd is :0.000


## 7. Conclusion 2

As we can see in the chi-squared test of the a and c versions, they are not statisticaly significantly different.

The rest is statistically significant (less than 'α / 6 = 0.008333'), but the versions B and C and worse in performance, so it is not interesting with our case.

In [187]:
pvalue_ac

0.464842123017306

## 8. Additional metrics

We see that the Drop-Off rate for A version is 10% and for C version 12%

Similarly, the Homepage-Return rate is 5% for A version and 4.5% for C version

In [188]:
loss_a = 1 - (1 - 0.098) * (1 - 0.052)
loss_c = 1 - (1 - 0.124) * (1 - 0.046)
print(f'loss for A version is {loss_a}')
print(f'loss for C version is {loss_c}')


loss for A version is 0.14490400000000003
loss for C version is 0.164296


In [232]:
loss = [
          512 * 0.145
        , 527 * 0.164]
no_loss = [
      512 * 0.855
    , 527 * 0.836]

lost_customers = pd.DataFrame([loss, no_loss], columns = ['a', 'c'], index = ["loss", "no-loss"])

lost_customers_ratio = lost_customers.copy()
lost_customers_ratio.loc['loss/100'] = lost_customers_ratio.iloc[0] / lost_customers_ratio.iloc[1] * 100
lost_customers_ratio

,a,c
loss,74.240,86.428
no-loss,437.760,440.572
loss/100,16.959,19.617


In [233]:
statistic_lc, pvalue_lc, dof_lc, freq_lc = stats.chi2_contingency(lost_customers)
freq_lc = pd.DataFrame(freq_lc, columns = ['a', 'c'], index = ['click', 'no-click'])
pvalue_lc

0.44663435530727624

## 8. Final conclusion

Unknown. The version A has statistically insignificant better CTR and version C has statistically insignificant higher drop out.